In [59]:
import os,sys
import logging, time
from pprint import pprint, pformat
import importlib
import dateparser
from datetime import timedelta, datetime
import pandas as pd
import numpy as np
import qgrid
import plotly.offline as offline
import plotly.tools as tools, plotly.graph_objs as go
from docs.config import *
from docs.data import *
import app
from app import freqtostr, strtofreq, pertostr
from app.common.timer import Timer
from app.common.utils import utc_datetime as now, utc_dtdate
from app.bot import scanner, candles, signals, trade, strategy

# Conf
log = logging.getLogger("testing")
pd.set_option("display.max_columns", 25)
pd.set_option("display.width", 2000)
app.set_db(["localhost", "45.79.176.125"][0])
db = app.get_db()

# Conf
pair='ONTBTC'
freq=3600
span = "48 hours ago utc"
periods = 48
macd_sh = 12
macd_ln = 26

candles.update([pair], freqtostr[freq], start=span, force=True)
app.bot.dfc = candles.merge_new(pd.DataFrame(), [pair], span=(now()-dateparser.parse(span))*3)
asset = app.bot.dfc.loc[pair,freq]
macd = signals.macd(asset, macd_sh, macd_ln).tail(periods)
asset = asset.tail(periods)

diff = macd['macd_diff']

buy_vol = macd['volume'] * macd['buy_ratio']
vema = buy_vol.ewm(span=3).mean() #.pct_change()

# Offline plot
fig = tools.make_subplots(
    rows=4, cols=1,
    specs=[ [{}], [{}], [{}], [{}] ], 
    shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.001)
fig.append_trace(go.Scatter(x=macd.index,y=macd['close']), 1, 1)
fig.append_trace(go.Bar(x=macd.index, y=macd['normalized'], name="MACD_diff (normalized)"), 2, 1)
fig.append_trace(go.Scatter(x=macd.index, y=vema, name="Buy Volume"), 3, 1)
fig.append_trace(go.Scatter(x=macd.index, y=macd['buy_ratio'], name="Buy Ratio"), 4, 1)

fig['layout'].update(title='{} MACD'.format(pair))

offline.plot(fig, filename="myplot.html")

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]



'file:///Users/seanestey/coding/coincruncher/myplot.html'

In [2]:
histos=[]
is_pos = None
current=[]
s = macd['normalized'].dropna()

for k,v in s.items():
    if is_pos is None:
        if v >= 0:
            is_pos = True
        else:
            is_pos = False  
    
    if is_pos == True:
        if v >= 0:
            current.append({k:v})
        else:
            histos.append(current.copy())
            current=[{k:v}]
            is_pos=False        
    elif is_pos == False:
        if v < 0:
            current.append({k:v})
        else:
            histos.append(current.copy())
            current=[{k:v}]
            is_pos=True
            
        

        

In [52]:
#df['elderly'] = np.where(df['age']>=50, 'yes', 'no')
macd['pos_neg'] = np.where(macd['normalized'] >=0, True, False)
mypos = macd[macd['pos_neg'] == True]

i = mypos.index[0]
j = mypos.index[1]

(j-i).seconds

300